In [1]:
import os
import sys
from pathlib import Path
import logging
import time
import pandas as pd
import geopandas as gpd
import seaborn
import dask
import numpy as np
import dask.dataframe as dd
from dask.distributed import Client

sys.path.insert(0, '..')
import src.hotspot_utils as util 
import src.process_nearest_hotspots as nearest_process

In [2]:
logging.basicConfig(
    format='%(asctime)s [%(levelname)s] %(name)s - %(message)s',
    level=logging.DEBUG,
    datefmt='%Y-%m-%d %H:%M:%S',
    stream=sys.stdout,
)
_LOG = logging.getLogger(__name__)

In [3]:
client = Client(processes=False)

In [4]:
# This is the output directory outputs and itermediary files from this notebook examples will be stored.
outdir = Path("/home/jovyan/s3vt_dask/s3vtdata/workdir")

# Processing Parameter used in Sub-setting Spatial Extent and Temporal Range for Area of Interest
##### The FRP data from nasa, esa, eumetsat and landgate are merged, sub-setted and neareast hotspots csv files are generated based on the parameters in `processing_parameters`  
##### The parameter `chunks` in blocking FRP data to enable multi-processing. If you encounter memory issues then higher the number.
##### The parameter `compare_field` is the name of column that will be used to generate nearest hotspots, the data in the column should be of datetime type.

In [5]:
processing_parameters = {
    "nasa_frp": "s3://s3vtaustralia/nasa_hotspots_gdf.geojson",
    "esa_frp": "s3://s3vtaustralia/s3vt_hotspots.geojson",
    "eumetsat_frp": "s3://s3vtaustralia/s3vt_eumetsat_hotspots.geojson",
    "landgate_frp": "s3://s3vtaustralia/landgate_hotspots_gdf.geojson",
    "dea_frp": None,
    "lon_west": 147.0,
    "lat_south": -38.0,
    "lon_east": 154.0,
    "lat_north": -27.,
    "start_date": "2019-11-01",
    "end_date": "2020-10-08",
    "start_time": "21:00",
    "end_time": "3:00",
    "chunks": 300,
    "outdir": outdir,
    "compare_field": "solar_day",  # solar_day or solar_night
    "swath_config_file": Path("/home/jovyan/s3vt_dask/s3vtdata/configs/s3vtconfig.yaml")
}

In [ ]:
# This is to generate nearest .csv files. If .csv files already exists then skip this process. Takes around ~5-6 hours in this sandbox environment with 2-core and 16 GB RAM
nearest_hotspots_product_files = nearest_process.process_nearest_points(**processing_parameters)

## Nearest Hotspots DataFrame merged from neareast hotspots csv files

In [6]:
# csv directory is where nearest hotspots csv files are stored. 
csv_directory = outdir
# This is read all the .csv files if name starts with `nearest_points` and ends with `compare_field` value from processing parameters.
nearest_hotspots_csv_files = [
    fp for fp in csv_directory.iterdir()
    if (fp.name.startswith("nearest_points"))
    and (fp.name.endswith(f"{processing_parameters['compare_field']}.csv"))
]

In [7]:
# nearest points csv files that will be used analysis from here on.
nearest_hotspots_csv_files

[PosixPath('/home/jovyan/s3vt_dask/s3vtdata/workdir/nearest_points_NOAA 20_VIIRS_NASA2.0NRT_solar_day.csv'),
 PosixPath('/home/jovyan/s3vt_dask/s3vtdata/workdir/nearest_points_SENTINEL_3B_SLSTR_EUMETSAT_solar_day.csv'),
 PosixPath('/home/jovyan/s3vt_dask/s3vtdata/workdir/nearest_points_SUOMI NPP_VIIRS_LANDGATE_solar_day.csv'),
 PosixPath('/home/jovyan/s3vt_dask/s3vtdata/workdir/nearest_points_AQUA_MODIS_LANDGATE_solar_day.csv'),
 PosixPath('/home/jovyan/s3vt_dask/s3vtdata/workdir/nearest_points_SENTINEL_3A_SLSTR_EUMETSAT_solar_day.csv'),
 PosixPath('/home/jovyan/s3vt_dask/s3vtdata/workdir/nearest_points_SENTINEL_3A_SLSTR_ESA_solar_day.csv'),
 PosixPath('/home/jovyan/s3vt_dask/s3vtdata/workdir/nearest_points_SUOMI NPP_VIIRS_NASA1_solar_day.csv'),
 PosixPath('/home/jovyan/s3vt_dask/s3vtdata/workdir/nearest_points_AQUA_MODIS_NASA6.03_solar_day.csv'),
 PosixPath('/home/jovyan/s3vt_dask/s3vtdata/workdir/nearest_points_TERRA_MODIS_LANDGATE_solar_day.csv'),
 PosixPath('/home/jovyan/s3vt_dask/

In [8]:
# returns a dask DataFrame with index set at column `compare_field` from processing_parameters.
nearest_points_ddf = util.csv_to_dataframe(nearest_hotspots_csv_files, processing_parameters["compare_field"])

In [9]:
nearest_points_ddf.head()

,Unnamed: 0,latitude,longitude,satellite,sensor,confidence,power,datetime,satellite_sensor_product,geometry,...,2_power,2_datetime,2_solar_day,2_satellite_sensor_product,2_geometry,2_solar_night,dist,dist_m,timedelta,count
solar_day,,,,,,,,,,,,,,,,,,,,,
2020-03-10 00:03:15,0,-34.01535,147.81364,NOAA 20,VIIRS,50.0,0.9,2020-03-09 14:12:00,NOAA 20_VIIRS_NASA2.0NRT,POINT (147.81364 -34.01535),...,0.90,2020-03-09 14:12:00,2020-03-10 00:03:15,NOAA 20_VIIRS_NASA2.0NRT,POINT (147.81364 -34.01535),2020-03-09 12:03:15,0.000000,0.000000,00:00:00,1
2020-03-10 00:03:15,39716,-34.01535,147.81364,NOAA 20,VIIRS,50.0,0.9,2020-03-09 14:12:00,NOAA 20_VIIRS_NASA2.0NRT,POINT (147.81364 -34.01535),...,0.26,2020-03-10 12:17:55,2020-03-10 22:08:59,SENTINEL_3B_SLSTR_EUMETSAT,POINT (147.7693696269182 -33.47744191547758),2020-03-10 10:08:59,0.539727,59804.434005,22:05:55,1
2020-03-10 00:03:15,35549,-34.01535,147.81364,NOAA 20,VIIRS,50.0,0.9,2020-03-09 14:12:00,NOAA 20_VIIRS_NASA2.0NRT,POINT (147.81364 -34.01535),...,0.21,2020-03-10 12:55:52,2020-03-10 22:50:46,SENTINEL_3A_SLSTR_EUMETSAT,POINT (148.7281626244975 -37.77098190463744),2020-03-10 10:50:46,3.865375,424808.005858,22:43:52,1
2020-03-10 00:03:15,14477,-34.01535,147.81364,NOAA 20,VIIRS,50.0,0.9,2020-03-09 14:12:00,NOAA 20_VIIRS_NASA2.0NRT,POINT (147.81364 -34.01535),...,0.88,2020-03-09 15:01:00,2020-03-10 00:53:10,SUOMI NPP_VIIRS_NASA1,POINT (148.045395 -34.560627),2020-03-09 12:53:10,0.592484,64139.655429,00:49:00,1
2020-03-10 00:03:15,31826,-34.01535,147.81364,NOAA 20,VIIRS,50.0,0.9,2020-03-09 14:12:00,NOAA 20_VIIRS_NASA2.0NRT,POINT (147.81364 -34.01535),...,0.27,2020-03-10 12:17:55,2020-03-10 22:08:59,SENTINEL_3B_SLSTR_ESA,POINT (147.7693696269182 -33.47744191547758),2020-03-10 10:08:59,0.539727,59804.434005,22:05:55,1


# Results
## Co-occurrence metrics

In [10]:
region_alias = "nsw"
output_directory = processing_parameters["outdir"]
comparison_prefix = (
    f"{processing_parameters['compare_field']}"
    f"_{processing_parameters['start_date'].replace('-', '')}"
    f"_{processing_parameters['end_date'].replace('-', '')}"
    f"_{region_alias}"
)

In [11]:
# set the nearest distance threshold between two hotspots to confine the analysis within the distance threshold.  
dist_threshold = 5000  # units in meters

In [12]:
nearest_ddf_dist_subset = client.persist(nearest_points_ddf[nearest_points_ddf["dist_m"] < dist_threshold])

In [13]:
# Count of hotspot matches < dist_threshold
numerator = util.dask_pivot_table(
    nearest_ddf_dist_subset,
    index="2_satellite_sensor_product",
    column="satellite_sensor_product",
    values="count",
    aggfunc="count"
).compute()

In [14]:
numerator.astype(int).to_csv(output_directory.joinpath(f"{comparison_prefix}_matches_{dist_threshold}.csv"))
numerator.astype(int)

satellite_sensor_product,NOAA 20_VIIRS_NASA2.0NRT,SUOMI NPP_VIIRS_LANDGATE,SUOMI NPP_VIIRS_NASA1,NOAA 20_VIIRS_LANDGATE,SENTINEL_3B_SLSTR_EUMETSAT,SENTINEL_3B_SLSTR_ESA,SENTINEL_3A_SLSTR_EUMETSAT,SENTINEL_3A_SLSTR_ESA,TERRA_MODIS_NASA6.03,TERRA_MODIS_LANDGATE,AQUA_MODIS_LANDGATE,AQUA_MODIS_NASA6.03
2_satellite_sensor_product,,,,,,,,,,,,
NOAA 20_VIIRS_NASA2.0NRT,14477,23769,4784,25258,733,442,684,358,110,716,1135,94
SUOMI NPP_VIIRS_NASA1,4655,9608,6145,6975,328,461,267,385,110,398,696,84
SUOMI NPP_VIIRS_LANDGATE,9452,29407,4669,18538,559,312,596,341,107,641,1091,93
SENTINEL_3B_SLSTR_ESA,869,1161,781,1281,1008,1565,168,236,121,411,189,13
SENTINEL_3B_SLSTR_EUMETSAT,1513,2704,601,2489,2166,1008,426,166,83,432,252,13
NOAA 20_VIIRS_LANDGATE,11714,21784,4043,26816,616,384,620,324,89,622,1063,87
SENTINEL_3A_SLSTR_EUMETSAT,1124,2183,435,1903,596,254,2257,867,81,534,155,13
TERRA_MODIS_NASA6.03,451,1168,410,734,232,318,177,280,317,748,136,18
TERRA_MODIS_LANDGATE,1597,3811,765,2654,531,482,669,463,297,1865,357,23


In [15]:
# Count of hotspot matches - total  
denominator = util.dask_pivot_table(
        nearest_points_ddf,
        index="2_satellite_sensor_product",
        column="satellite_sensor_product",
        values="count",
        aggfunc="count",
    ).compute()

In [16]:
denominator.astype(int).to_csv(output_directory.joinpath(f"{comparison_prefix}_matches_count.csv"))
denominator.astype(int)

satellite_sensor_product,NOAA 20_VIIRS_NASA2.0NRT,SUOMI NPP_VIIRS_LANDGATE,SUOMI NPP_VIIRS_NASA1,NOAA 20_VIIRS_LANDGATE,SENTINEL_3B_SLSTR_ESA,SENTINEL_3B_SLSTR_EUMETSAT,SENTINEL_3A_SLSTR_EUMETSAT,SENTINEL_3A_SLSTR_ESA,TERRA_MODIS_NASA6.03,TERRA_MODIS_LANDGATE,AQUA_MODIS_LANDGATE,AQUA_MODIS_NASA6.03
2_satellite_sensor_product,,,,,,,,,,,,
NOAA 20_VIIRS_NASA2.0NRT,14477,29143,5949,26541,1496,2097,2142,1402,310,1819,1150,97
SENTINEL_3B_SLSTR_EUMETSAT,4821,8335,2315,8200,1016,2166,724,284,101,573,504,37
SENTINEL_3A_SLSTR_EUMETSAT,4167,8456,2100,6804,520,1145,2257,874,100,651,417,28
SUOMI NPP_VIIRS_NASA1,6058,10111,6145,9050,1562,1022,898,1517,266,987,704,84
SENTINEL_3B_SLSTR_ESA,3723,5336,3615,5156,1565,1022,297,483,159,590,506,49
SUOMI NPP_VIIRS_LANDGATE,13623,29407,6069,26086,1529,2000,2089,1502,316,1845,1153,98
SENTINEL_3A_SLSTR_ESA,3147,4944,3364,4410,796,527,898,1517,155,565,338,40
TERRA_MODIS_NASA6.03,5106,9890,4734,7749,1095,710,625,1055,317,1078,690,93
TERRA_MODIS_LANDGATE,10733,23114,5290,19493,1300,1474,1729,1236,315,1865,1074,96


In [17]:
# Difference of matched points closer than 5000m
difference = denominator - numerator

In [18]:
difference.astype(int).to_csv(output_directory.joinpath(f"{comparison_prefix}_count_difference.csv"))
difference.astype(int)

satellite_sensor_product,NOAA 20_VIIRS_NASA2.0NRT,SUOMI NPP_VIIRS_LANDGATE,SUOMI NPP_VIIRS_NASA1,NOAA 20_VIIRS_LANDGATE,SENTINEL_3B_SLSTR_ESA,SENTINEL_3B_SLSTR_EUMETSAT,SENTINEL_3A_SLSTR_EUMETSAT,SENTINEL_3A_SLSTR_ESA,TERRA_MODIS_NASA6.03,TERRA_MODIS_LANDGATE,AQUA_MODIS_LANDGATE,AQUA_MODIS_NASA6.03
2_satellite_sensor_product,,,,,,,,,,,,
NOAA 20_VIIRS_NASA2.0NRT,0,5374,1165,1283,1054,1364,1458,1044,200,1103,15,3
SENTINEL_3B_SLSTR_EUMETSAT,3308,5631,1714,5711,8,0,298,118,18,141,252,24
SENTINEL_3A_SLSTR_EUMETSAT,3043,6273,1665,4901,266,549,0,7,19,117,262,15
SUOMI NPP_VIIRS_NASA1,1403,503,0,2075,1101,694,631,1132,156,589,8,0
SENTINEL_3B_SLSTR_ESA,2854,4175,2834,3875,0,14,129,247,38,179,317,36
SUOMI NPP_VIIRS_LANDGATE,4171,0,1400,7548,1217,1441,1493,1161,209,1204,62,5
SENTINEL_3A_SLSTR_ESA,2586,3933,2738,3490,442,271,25,0,29,117,213,23
TERRA_MODIS_NASA6.03,4655,8722,4324,7015,777,478,448,775,0,330,554,75
TERRA_MODIS_LANDGATE,9136,19303,4525,16839,818,943,1060,773,18,0,717,73


In [19]:
# Percentage of matched points closer than dist_threshold
percentage = (numerator / denominator) * 100
percentage = np.round(percentage, 2)

In [20]:
percentage.to_csv(output_directory.joinpath(f"{comparison_prefix}_percentage.csv"))
percentage

satellite_sensor_product,NOAA 20_VIIRS_NASA2.0NRT,SUOMI NPP_VIIRS_LANDGATE,SUOMI NPP_VIIRS_NASA1,NOAA 20_VIIRS_LANDGATE,SENTINEL_3B_SLSTR_EUMETSAT,SENTINEL_3B_SLSTR_ESA,SENTINEL_3A_SLSTR_EUMETSAT,SENTINEL_3A_SLSTR_ESA,TERRA_MODIS_NASA6.03,TERRA_MODIS_LANDGATE,AQUA_MODIS_LANDGATE,AQUA_MODIS_NASA6.03
2_satellite_sensor_product,,,,,,,,,,,,
NOAA 20_VIIRS_NASA2.0NRT,100.00,81.56,80.42,95.17,34.95,29.55,31.93,25.53,35.48,39.36,98.70,96.91
SUOMI NPP_VIIRS_NASA1,76.84,95.03,100.00,77.07,32.09,29.51,29.73,25.38,41.35,40.32,98.86,100.00
SUOMI NPP_VIIRS_LANDGATE,69.38,100.00,76.93,71.06,27.95,20.41,28.53,22.70,33.86,34.74,94.62,94.90
SENTINEL_3B_SLSTR_ESA,23.34,21.76,21.60,24.84,98.63,100.00,56.57,48.86,76.10,69.66,37.35,26.53
SENTINEL_3B_SLSTR_EUMETSAT,31.38,32.44,25.96,30.35,100.00,99.21,58.84,58.45,82.18,75.39,50.00,35.14
NOAA 20_VIIRS_LANDGATE,84.72,74.87,65.88,100.00,30.08,24.58,28.73,21.36,28.25,34.01,90.62,88.78
SENTINEL_3A_SLSTR_EUMETSAT,26.97,25.82,20.71,27.97,52.05,48.85,100.00,99.20,81.00,82.03,37.17,46.43
TERRA_MODIS_NASA6.03,8.83,11.81,8.66,9.47,32.68,29.04,28.32,26.54,100.00,69.39,19.71,19.35
TERRA_MODIS_LANDGATE,14.88,16.49,14.46,13.62,36.02,37.08,38.69,37.46,94.29,100.00,33.24,23.96


In [21]:
# Maximum time between match points < dist_threshold
timemax = util.pandas_pivot_table(
    nearest_ddf_dist_subset.compute(),
    index=["satellite_sensor_product"],
    columns=["2_satellite_sensor_product"],
    values=["timedelta"],
    aggfunc={"timedelta": np.max}
    
)

In [22]:
timemax.to_csv(output_directory.joinpath(f"{comparison_prefix}_max_time_matched_points.csv"))
timemax

timedelta                      \
2_satellite_sensor_product AQUA_MODIS_LANDGATE AQUA_MODIS_NASA6.03   
satellite_sensor_product                                             
AQUA_MODIS_LANDGATE                   00:15:00            00:11:00   
AQUA_MODIS_NASA6.03                   00:11:00            00:00:00   
NOAA 20_VIIRS_LANDGATE                01:18:00            01:06:00   
NOAA 20_VIIRS_NASA2.0NRT              22:17:00            01:30:00   
SENTINEL_3A_SLSTR_ESA                 22:00:43            21:18:04   
SENTINEL_3A_SLSTR_EUMETSAT            22:00:43            21:18:04   
SENTINEL_3B_SLSTR_ESA                 22:25:50            22:00:14   
SENTINEL_3B_SLSTR_EUMETSAT            22:03:14            22:00:14   
SUOMI NPP_VIIRS_LANDGATE              01:11:00            00:59:00   
SUOMI NPP_VIIRS_NASA1                 23:08:00            22:51:00   
TERRA_MODIS_LANDGATE                  21:48:00            21:40:00   
TERRA_MODIS_NASA6.03                  21:39:00            21:32:00   

                                                                            \
2_satellite_sensor_product NOAA 20_VIIRS_LANDGATE NOAA 20_VIIRS_NASA2.0NRT   
satellite_sensor_product                                                     
AQUA_MODIS_LANDGATE                      01:25:00                 01:49:00   
AQUA_MODIS_NASA6.03                      01:06:00                 01:17:00   
NOAA 20_VIIRS_LANDGATE                   01:44:00                 23:42:00   
NOAA 20_VIIRS_NASA2.0NRT                 23:42:00                 00:00:00   
SENTINEL_3A_SLSTR_ESA                    22:24:15                 22:44:02   
SENTINEL_3A_SLSTR_EUMETSAT               22:24:15                 22:44:02   
SENTINEL_3B_SLSTR_ESA                    22:20:11                 22:53:39   
SENTINEL_3B_SLSTR_EUMETSAT               22:35:22                 22:53:39   
SUOMI NPP_VIIRS_LANDGATE                 22:56:00                 23:00:00   
SUOMI NPP_VIIRS_NASA1                    22:58:00                 22:54:00   
TERRA_MODIS_LANDGATE                     23:08:00                 23:35:00   
TERRA_MODIS_NASA6.03                     22:48:00                 23:12:00   

                                                                             \
2_satellite_sensor_product SENTINEL_3A_SLSTR_ESA SENTINEL_3A_SLSTR_EUMETSAT   
satellite_sensor_product                                                      
AQUA_MODIS_LANDGATE                     22:00:43                   22:00:43   
AQUA_MODIS_NASA6.03                     21:18:04                   21:18:04   
NOAA 20_VIIRS_LANDGATE                  22:24:15                   22:24:15   
NOAA 20_VIIRS_NASA2.0NRT                22:59:23                   22:44:02   
SENTINEL_3A_SLSTR_ESA                   00:00:01                   00:00:01   
SENTINEL_3A_SLSTR_EUMETSAT              00:00:01                   00:00:01   
SENTINEL_3B_SLSTR_ESA                   00:40:09                   00:40:05   
SENTINEL_3B_SLSTR_EUMETSAT              00:40:05                   00:40:05   
SUOMI NPP_VIIRS_LANDGATE                22:29:57                   22:46:07   
SUOMI NPP_VIIRS_NASA1                   22:54:48                   22:54:48   
TERRA_MODIS_LANDGATE                    01:25:57                   01:42:36   
TERRA_MODIS_NASA6.03                    01:05:00                   01:05:00   

                                                                             \
2_satellite_sensor_product SENTINEL_3B_SLSTR_ESA SENTINEL_3B_SLSTR_EUMETSAT   
satellite_sensor_product                                                      
AQUA_MODIS_LANDGATE                     22:25:50                   22:03:14   
AQUA_MODIS_NASA6.03                     22:00:14                   22:00:14   
NOAA 20_VIIRS_LANDGATE                  22:20:11                   22:35:22   
NOAA 20_VIIRS_NASA2.0NRT                23:01:29                   23:01:29   
SENTINEL_3A_SLSTR_ESA                   00:40:09                   00:40:05 

In [23]:
# Minimum time between match points < dist_threshold
timemin = util.pandas_pivot_table(
    nearest_ddf_dist_subset.compute(),
    index=["satellite_sensor_product"],
    columns=["2_satellite_sensor_product"],
    values=["timedelta"],
    aggfunc={"timedelta": np.min}
    
)

In [24]:
timemin.to_csv(output_directory.joinpath(f"{comparison_prefix}_min_time_matched_points.csv"))
timemin

timedelta                      \
2_satellite_sensor_product AQUA_MODIS_LANDGATE AQUA_MODIS_NASA6.03   
satellite_sensor_product                                             
AQUA_MODIS_LANDGATE                   00:00:00            00:00:00   
AQUA_MODIS_NASA6.03                   00:00:00            00:00:00   
NOAA 20_VIIRS_LANDGATE                00:00:00            00:03:00   
NOAA 20_VIIRS_NASA2.0NRT              00:04:00            00:01:00   
SENTINEL_3A_SLSTR_ESA                 20:49:52            20:50:10   
SENTINEL_3A_SLSTR_EUMETSAT            20:37:58            20:51:56   
SENTINEL_3B_SLSTR_ESA                 20:39:55            20:35:55   
SENTINEL_3B_SLSTR_EUMETSAT            19:59:08            20:35:55   
SUOMI NPP_VIIRS_LANDGATE              00:01:00            00:04:00   
SUOMI NPP_VIIRS_NASA1                 00:00:00            00:01:00   
TERRA_MODIS_LANDGATE                  21:13:00            21:21:00   
TERRA_MODIS_NASA6.03                  21:31:00            21:28:00   

                                                                            \
2_satellite_sensor_product NOAA 20_VIIRS_LANDGATE NOAA 20_VIIRS_NASA2.0NRT   
satellite_sensor_product                                                     
AQUA_MODIS_LANDGATE                      00:01:00                 00:04:00   
AQUA_MODIS_NASA6.03                      00:03:00                 00:01:00   
NOAA 20_VIIRS_LANDGATE                   00:00:00                 00:00:00   
NOAA 20_VIIRS_NASA2.0NRT                 00:00:00                 00:00:00   
SENTINEL_3A_SLSTR_ESA                    19:59:28                 00:57:25   
SENTINEL_3A_SLSTR_EUMETSAT               01:24:59                 00:49:51   
SENTINEL_3B_SLSTR_ESA                    20:25:15                 00:46:35   
SENTINEL_3B_SLSTR_EUMETSAT               20:09:30                 00:49:02   
SUOMI NPP_VIIRS_LANDGATE                 00:44:00                 00:40:00   
SUOMI NPP_VIIRS_NASA1                    00:41:00                 00:46:00   
TERRA_MODIS_LANDGATE                     20:11:00                 00:20:00   
TERRA_MODIS_NASA6.03                     20:17:00                 00:30:00   

                                                                             \
2_satellite_sensor_product SENTINEL_3A_SLSTR_ESA SENTINEL_3A_SLSTR_EUMETSAT   
satellite_sensor_product                                                      
AQUA_MODIS_LANDGATE                     20:35:52                   20:27:59   
AQUA_MODIS_NASA6.03                     20:50:10                   20:51:56   
NOAA 20_VIIRS_LANDGATE                  19:57:28                   01:24:59   
NOAA 20_VIIRS_NASA2.0NRT                00:57:25                   00:49:51   
SENTINEL_3A_SLSTR_ESA                   00:00:00                   00:00:00   
SENTINEL_3A_SLSTR_EUMETSAT              00:00:00                   00:00:00   
SENTINEL_3B_SLSTR_ESA                   00:38:54                   00:38:54   
SENTINEL_3B_SLSTR_EUMETSAT              00:38:54                   00:38:49   
SUOMI NPP_VIIRS_LANDGATE                20:10:30                   20:06:13   
SUOMI NPP_VIIRS_NASA1                   01:46:40                   01:46:40   
TERRA_MODIS_LANDGATE                    00:00:40                   00:00:09   
TERRA_MODIS_NASA6.03                    00:00:01                   00:00:01   

                                                                             \
2_satellite_sensor_product SENTINEL_3B_SLSTR_ESA SENTINEL_3B_SLSTR_EUMETSAT   
satellite_sensor_product                                                      
AQUA_MODIS_LANDGATE                     20:37:55                   19:49:08   
AQUA_MODIS_NASA6.03                     20:35:55                   20:35:55   
NOAA 20_VIIRS_LANDGATE                  20:24:16                   20:09:30   
NOAA 20_VIIRS_NASA2.0NRT                00:46:35                   00:49:02   
SENTINEL_3A_SLSTR_ESA                   00:38:54                   00:38:54 

In [25]:
# Average distance (m) between matched points < dist_threshold
averagedist = util.dask_pivot_table(
    nearest_ddf_dist_subset,
    index="2_satellite_sensor_product",
    column="satellite_sensor_product",
    values="dist_m",
    aggfunc="mean",
).compute()

In [26]:
averagedist = np.round(averagedist, 2)
averagedist.to_csv(output_directory.joinpath(f"{comparison_prefix}_average_distance_{dist_threshold}m.csv"))
averagedist

satellite_sensor_product,NOAA 20_VIIRS_NASA2.0NRT,SUOMI NPP_VIIRS_LANDGATE,SUOMI NPP_VIIRS_NASA1,NOAA 20_VIIRS_LANDGATE,SENTINEL_3B_SLSTR_EUMETSAT,SENTINEL_3B_SLSTR_ESA,SENTINEL_3A_SLSTR_EUMETSAT,SENTINEL_3A_SLSTR_ESA,TERRA_MODIS_NASA6.03,TERRA_MODIS_LANDGATE,AQUA_MODIS_LANDGATE,AQUA_MODIS_NASA6.03
2_satellite_sensor_product,,,,,,,,,,,,
NOAA 20_VIIRS_NASA2.0NRT,0.00,328.56,277.46,167.17,1109.05,1320.60,1019.80,1082.21,851.59,1002.95,669.97,350.12
SUOMI NPP_VIIRS_NASA1,275.68,228.91,0.00,296.05,1453.33,1311.38,1147.44,1034.79,773.43,898.89,704.81,420.13
SUOMI NPP_VIIRS_LANDGATE,311.71,0.00,285.48,233.19,1124.81,1384.02,1017.01,1119.69,776.39,902.73,622.38,396.04
SENTINEL_3B_SLSTR_ESA,1122.11,1121.29,1098.07,1173.66,2.10,0.00,683.11,669.57,636.97,859.24,1622.07,976.46
SENTINEL_3B_SLSTR_EUMETSAT,983.10,929.98,1175.06,1032.77,0.00,1.62,715.70,684.70,639.55,849.83,1469.94,976.55
NOAA 20_VIIRS_LANDGATE,163.51,251.24,354.52,0.00,1155.02,1421.10,1026.72,1232.25,878.74,991.44,598.04,321.44
SENTINEL_3A_SLSTR_EUMETSAT,763.30,827.49,814.11,798.56,1001.92,1036.41,0.00,3.12,567.95,778.07,921.46,639.79
TERRA_MODIS_NASA6.03,1045.56,1129.20,955.23,1012.45,1404.80,1403.36,1045.93,989.78,0.00,550.45,1132.48,816.36
TERRA_MODIS_LANDGATE,1122.08,1030.12,1081.01,1032.31,1347.41,1341.93,1092.62,1028.90,340.75,0.00,1043.41,833.51


In [27]:
client.close() # close dask.distributed client